# Warsztaty Python w Data Science

---
## Text Mining - część 2 z 3  

### Biblioteki NLP
#### - Scikit-Learn
#### - NLTK

#### - Gensim
#### - Spacy



### Scikit-Learn
- Potężna biblioteka do nauczania maszynowego
- Posiada wiele narzędzi do obróbki statystycznej danych tekstowych

### NLTK
- Biblioteka do przetwarzania języka naturalnego
- Posiada wiele narzędzi do obróbki statystycznej danych tekstowych
- KORPUSY

### Spacy
- Dużo wytrenowanych modeli statystycznych do języka
- Chyba najlepsze modele do języka polskiego - morfologia, tagging (części mowy)

### Gensim
- Zbiór najnowszych algorytmów do obróbki danych tekstowych
- Word2Vec, CBOW etc. etc.

---
## Zadanie 1

Podać wyrazy z korpusu o największym `TF-IDF` z pominięciem wyrazów które wystepują _**raz**_.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
myvocabulary = ['life', 'learning']
corpus = {1: "The game of life is a game of everlasting learning", 
          2: "The unexamined life is not worth living", 
          3: "Never stop learning"}
tfidf = TfidfVectorizer(vocabulary = myvocabulary, ngram_range = (1,3))
tfs = tfidf.fit_transform(corpus.values())

feature_names = tfidf.get_feature_names()
corpus_index = [n for n in corpus]
import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
print(df)

                 1    2    3
life      0.707107  1.0  0.0
learning  0.707107  0.0  1.0


In [7]:
import re

data = pd.read_csv('data\gumtree-2021-03-09.csv', sep='|')
columns = list(data.columns)
columns[0] = "Index"
data.columns=columns
data.set_index('Index', drop=True, inplace=True)
data.drop(["title", "url"], axis=1,inplace=True)

def no_tags(s):
    return re.sub(r'<[^<]+?>','',str(s))

data["description"] = data["description"].apply(no_tags)

In [8]:
data.head()

,description
Index,
0,Na sprzedaż piękna kawalerka o powierzchni 24 ...
1,"Mieszkanie dwupokojowe,własnościowe z 1971 r n..."
2,OPIS INWESTYCJI\n===============\nPOWER INVEST...
3,Bezpośrednio od dewelopera- brak prowizji 0%- ...
4,Na sprzedaż ekskluzywne mieszkanie dwupokojowe...


In [9]:
import gzip
import sys
import re

f = gzip.open('data/odm.txt.gz', 'rt', encoding='utf-8')
dictionary = {}

for x in f:
    t = x.strip().split(',')
    tt = [ x.strip().lower() for x in t]
    for w in tt[1:]: 
        dictionary[w]=tt[0]

def lematize(w):
    return dictionary.get(w,w)

In [10]:
import re

splitter = re.compile(r'[^ąąćęńłóóśśżżź\w]+')
isnumber = re.compile(r'[0-9]')

def preprocessing(opis):
    opis = str(opis)
    
    tokenized = splitter.split(opis)
    l = list(tokenized)
    l = [ x.lower() for x in l if len(x)>2 ]
    l = [ x for x in l if isnumber.search(x) is None ]
    l = [ lematize(x) for x in l ]
    return l

In [11]:
data["clean_description"] = data["description"].apply(lambda x: ' '.join(preprocessing(x)))

In [12]:
data.head()

,description,clean_description
Index,,
0,Na sprzedaż piękna kawalerka o powierzchni 24 ...,sprzedaż piękny kawalerka powierzchnia ostatni...
1,"Mieszkanie dwupokojowe,własnościowe z 1971 r n...",mieszkać dwupokojowy własnościowy pierwszy pię...
2,OPIS INWESTYCJI\n===============\nPOWER INVEST...,opis inwestycja power invest przyjemność zapre...
3,Bezpośrednio od dewelopera- brak prowizji 0%- ...,bezpośredni deweloper brak prowizja brak podat...
4,Na sprzedaż ekskluzywne mieszkanie dwupokojowe...,sprzedaż ekskluzywny mieszkać dwupokojowy powi...


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = {1: "The game of life is a game of everlasting learning", 
          2: "The unexamined life is not worth living", 
          3: "Never stop learning"}
tfidf = TfidfVectorizer(min_df=2)
tfs = tfidf.fit_transform(data["clean_description"])

feature_names = tfidf.get_feature_names()


In [14]:
print(list(feature_names[:50]))

['aby', 'aczkolwiek', 'adaptacja', 'adres', 'agd', 'agencja', 'agent', 'aktualność', 'aktualny', 'aktywny', 'al', 'alejka', 'aluzyjny', 'amator', 'amfiteatr', 'andrychowicznieruchomosci', 'aneks', 'antresola', 'antywłamaniowy', 'apartament', 'apartamentowiec', 'apteka', 'aranżacja', 'aranżacyjny', 'architektoniczny', 'architektura', 'arkadia', 'armatura', 'art', 'atmosfera', 'atrakcja', 'atrakcjekomunikacja', 'atrakcyjny', 'atut', 'aut', 'autobus', 'autobusowy', 'bagno', 'bajkowy', 'balance', 'balkon', 'bank', 'bar', 'bardzo', 'basen', 'baza', 'bazarek', 'bazia', 'bem', 'bemowo']


In [15]:
df = pd.DataFrame(tfs.toarray(), 
columns=tfidf.get_feature_names())

In [16]:
df

,aby,aczkolwiek,adaptacja,adres,agd,agencja,agent,aktualność,aktualny,aktywny,...,świetny,świeży,żaden,żerać,żerań,żerańogłoszenie,życzyć,żyto,żyć,żłobek
0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0
1,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.141596,0.0,0.000000,0.0
2,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.061946,0.0
3,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.093885,0.0
4,0.117810,0.0,0.0,0.000000,0.061704,0.0,0.0,0.0,0.059788,0.0,...,0.044939,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,0.099201,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.100689,0.0,...,0.000000,0.0,0.0,0.308448,0.0,0.12965,0.000000,0.0,0.000000,0.0
220,0.097644,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.099108,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0
221,0.073988,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.075098,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0
222,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0


---
# NLTK

In [17]:
import pprint
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [18]:
opis = data['description'][0]
opis

'Na sprzedaż piękna kawalerka o powierzchni 24 m2 na ostatnim piętrze 10 piętrowego bloku z oknem wychodzącym na spokojną stronę osiedla. Bardzo dobrze skomunikowane z centrum (tramwaje ,autobusy).W pobliżu znajduje się dobra infrastruktura: sklepy, apteka, szkoła, targowisko ( hala Banacha),oraz park szczęśliwicki (5 minut na piechotę).Mieszkanie słoneczne i bardzo ustawne ,budynek po wymianie windy i elektryki w częściach wspólnych.Serdecznie zapraszamy do kontaktu.'

In [19]:
print(word_tokenize(opis))

['Na', 'sprzedaż', 'piękna', 'kawalerka', 'o', 'powierzchni', '24', 'm2', 'na', 'ostatnim', 'piętrze', '10', 'piętrowego', 'bloku', 'z', 'oknem', 'wychodzącym', 'na', 'spokojną', 'stronę', 'osiedla', '.', 'Bardzo', 'dobrze', 'skomunikowane', 'z', 'centrum', '(', 'tramwaje', ',', 'autobusy', ')', '.W', 'pobliżu', 'znajduje', 'się', 'dobra', 'infrastruktura', ':', 'sklepy', ',', 'apteka', ',', 'szkoła', ',', 'targowisko', '(', 'hala', 'Banacha', ')', ',', 'oraz', 'park', 'szczęśliwicki', '(', '5', 'minut', 'na', 'piechotę', ')', '.Mieszkanie', 'słoneczne', 'i', 'bardzo', 'ustawne', ',', 'budynek', 'po', 'wymianie', 'windy', 'i', 'elektryki', 'w', 'częściach', 'wspólnych.Serdecznie', 'zapraszamy', 'do', 'kontaktu', '.']


In [20]:
pprint.pprint(sent_tokenize(opis))

['Na sprzedaż piękna kawalerka o powierzchni 24 m2 na ostatnim piętrze 10 '
 'piętrowego bloku z oknem wychodzącym na spokojną stronę osiedla.',
 'Bardzo dobrze skomunikowane z centrum (tramwaje ,autobusy).W pobliżu '
 'znajduje się dobra infrastruktura: sklepy, apteka, szkoła, targowisko ( hala '
 'Banacha),oraz park szczęśliwicki (5 minut na piechotę).Mieszkanie słoneczne '
 'i bardzo ustawne ,budynek po wymianie windy i elektryki w częściach '
 'wspólnych.Serdecznie zapraszamy do kontaktu.']


In [21]:
tokens = [ word_tokenize(sentence) for sentence in sent_tokenize(opis)]
for sentence in tokens:
    print(sentence)

['Na', 'sprzedaż', 'piękna', 'kawalerka', 'o', 'powierzchni', '24', 'm2', 'na', 'ostatnim', 'piętrze', '10', 'piętrowego', 'bloku', 'z', 'oknem', 'wychodzącym', 'na', 'spokojną', 'stronę', 'osiedla', '.']
['Bardzo', 'dobrze', 'skomunikowane', 'z', 'centrum', '(', 'tramwaje', ',', 'autobusy', ')', '.W', 'pobliżu', 'znajduje', 'się', 'dobra', 'infrastruktura', ':', 'sklepy', ',', 'apteka', ',', 'szkoła', ',', 'targowisko', '(', 'hala', 'Banacha', ')', ',', 'oraz', 'park', 'szczęśliwicki', '(', '5', 'minut', 'na', 'piechotę', ')', '.Mieszkanie', 'słoneczne', 'i', 'bardzo', 'ustawne', ',', 'budynek', 'po', 'wymianie', 'windy', 'i', 'elektryki', 'w', 'częściach', 'wspólnych.Serdecznie', 'zapraszamy', 'do', 'kontaktu', '.']


In [22]:
from urllib import request
url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')
raw[:75]

'\ufeffThe Project Gutenberg EBook of Crime and Punishment, by Fyodor Dostoevsky\r'

In [23]:
tokens = word_tokenize(raw)
text = nltk.Text(tokens)
text.collocation_list()

['Katerina Ivanovna',
 'Pyotr Petrovitch',
 'Pulcheria Alexandrovna',
 'Avdotya Romanovna',
 'Rodion Romanovitch',
 'Marfa Petrovna',
 'Sofya Semyonovna',
 'old woman',
 'Project Gutenberg-tm',
 'Porfiry Petrovitch',
 'Amalia Ivanovna',
 'great deal',
 'young man',
 'Nikodim Fomitch',
 'Ilya Petrovitch',
 'Project Gutenberg',
 'Andrey Semyonovitch',
 'Hay Market',
 'Dmitri Prokofitch',
 'Good heavens']

In [24]:
alice = nltk.corpus.gutenberg.fileids()[7]
al = nltk.corpus.gutenberg.words(alice)
al_text = nltk.Text(al)
al_text.collocation_list(25)

['Mock Turtle',
 'said Alice',
 'March Hare',
 'White Rabbit',
 'thought Alice',
 'golden key',
 'beautiful Soup',
 'white kid',
 'good deal',
 'kid gloves',
 'Mary Ann',
 'yer honour',
 'three gardeners',
 'play croquet',
 'Lobster Quadrille',
 'ootiful Soo',
 'great hurry',
 'old fellow',
 'trembling voice',
 'poor little',
 'next witness',
 'feet high',
 'poor Alice',
 'inches high',
 'young lady']

!pip install regex

In [25]:
import nltk

#opis="Ala ma kota, kto tam przyszedł"

tc = nltk.classify.textcat.TextCat() 
tc.guess_language(opis)


'pol'

# Spacy

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text, token.pos_, token.dep_)

In [30]:
import spacy
from spacy.lang.pl.examples import sentences 

nlp = spacy.load("pl_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)


Poczuł przyjemną woń mocnej kawy.


In [31]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Poczuł VERB ROOT
przyjemną ADJ amod
woń ADV obj
mocnej ADJ amod
kawy NOUN obj
. PUNCT punct


In [32]:
doc = nlp(opis)
print(doc.text)

Na sprzedaż piękna kawalerka o powierzchni 24 m2 na ostatnim piętrze 10 piętrowego bloku z oknem wychodzącym na spokojną stronę osiedla. Bardzo dobrze skomunikowane z centrum (tramwaje ,autobusy).W pobliżu znajduje się dobra infrastruktura: sklepy, apteka, szkoła, targowisko ( hala Banacha),oraz park szczęśliwicki (5 minut na piechotę).Mieszkanie słoneczne i bardzo ustawne ,budynek po wymianie windy i elektryki w częściach wspólnych.Serdecznie zapraszamy do kontaktu.


In [33]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Na ADP case
sprzedaż NOUN amod
piękna ADJ amod
kawalerka NOUN ROOT
o ADP case
powierzchni NOUN nmod
24 NUM nummod
m2 NOUN nmod
na ADP case
ostatnim ADJ amod
piętrze NOUN nmod
10 PRON amod
piętrowego ADJ amod
bloku NOUN nmod
z ADP case
oknem NOUN nmod
wychodzącym ADJ amod
na ADP case
spokojną ADJ amod
stronę NOUN obl:arg
osiedla NOUN nmod
. PUNCT punct
Bardzo ADV advmod
dobrze ADV advmod
skomunikowane ADJ amod
z ADP case
centrum NOUN obl
( PUNCT punct
tramwaje NOUN nsubj
, PUNCT punct
autobusy).W NOUN amod
pobliżu NOUN conj
znajduje VERB ROOT
się PRON expl:pv
dobra ADJ amod
infrastruktura NOUN nsubj
: ADJ nmod
sklepy NOUN obj
, PUNCT punct
apteka VERB conj
, PUNCT punct
szkoła NOUN conj
, PUNCT punct
targowisko ADJ conj
( PUNCT punct
hala NOUN appos
Banacha),oraz CCONJ nmod
park NOUN conj
szczęśliwicki ADJ amod
( PUNCT punct
5 NUM nmod
minut NOUN nmod
na ADP case
piechotę).Mieszkanie NOUN nmod
słoneczne ADJ amod
i CCONJ cc
bardzo ADV advmod
ustawne ADJ amod
, PUNCT punct
budynek NOUN co

---
# Gensim

## Word2Vec - model wektorowy słów w oparciu o sieci neuronowe (płytkie) - ale mówi się na to Deep Learning

### vec(“king”) - vec(“man”) + vec(“woman”) =~ vec(“queen”)

In [ ]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors from gensim-data


In [ ]:
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])[0]

In [ ]:
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

---